In [151]:
from itertools import combinations
from time import time
start_time = time()

f = open("input.txt", "r")
maze_map = list(map(list, f.read().split("\n")))
rows, cols = len(maze_map), len(maze_map[0])

In [152]:
# Part 1: Find the distance to the furthest point on the loop, starting at S
connections_map = {
    "|": [(0, -1), (0, 1)],
    "-": [(1, 0), (-1, 0)],
    "L": [(0, -1), (1, 0)],
    "J": [(0, -1), (-1, 0)],
    "7": [(0, 1), (-1, 0)],
    "F": [(0, 1), (1, 0)],
    ".": [],
    "S": [(0, -1), (0, 1), (1, 0), (-1, 0)]
}

"""
Utility Functions
"""

# Safe read char at index.
def in_bounds(x, y):
    return not (x < 0 or x >= cols or y < 0 or y >= rows)

def read_index(x, y):
    return "." if not in_bounds(x, y) else maze_map[y][x]

# See if characters at two locations are connected or not
def are_adjacent(x1, y1, x2, y2):
    return (x2 - x1, y2 - y1) in connections_map[read_index(x1, y1)] and (x1 - x2, y1 - y2) in connections_map[read_index(x2, y2)]

# Identify if there is a duplicate value in a list, and return said value
def find_duplicate(l):
    return None if len(l) <= 1 else l[0] if l[0] in l[1:] else find_duplicate(l[1:])

# Given a set of points an equal distance from the start of the loop, check if there is a duplicate (we connected to it twice), or if two of the points are connected to each other.
def loop_finished(point_set):
    duplicate_point = find_duplicate(point_set)
    if not duplicate_point is None:
        return [duplicate_point], duplicate_point[2]
    for (x1, y1, d1), (x2, y2, d2) in combinations(point_set, 2):
        if are_adjacent(x1, y1, x2, y2):
            return [(x1, y1, d1), (x2, y2, d2)], d1
    return None

"""
Execution
"""
start_time = time()

# Locate the start of the loop.
start_x, start_y = -1, -1
for i, row in enumerate(maze_map):
    for j, char in enumerate(row):
        if char == "S":
            start_x, start_y = j, i
            break


# Use a matrix to store the depth of each location in the maze
search_matrix = [[-1 for _ in row] for row in maze_map]

# Do a breath-first search to have a snapshot of all positions at a certain depth, so we can check for any double values or for any adjacent ones to signify a solution was found.
search_queue, next_queue = [], [(start_x, start_y, 0)]
while loop_finished(next_queue) is None and len(next_queue) > 0:
    search_queue, next_queue = next_queue, []
    for sx, sy, sd in search_queue:
        search_matrix[sy][sx] = sd
        expansions = connections_map[read_index(sx, sy)]
        for ex, ey in expansions:
            if are_adjacent(sx, sy, sx + ex, sy+ey) and search_matrix[sy + ey][sx + ex] == -1:
                next_queue.append((sx + ex, sy + ey, sd + 1))

loop_end, depth = loop_finished(next_queue)

print(f"Max Depth: {depth:,}")
print(f"P1 Runtime: {1000*(time() - start_time):.2f}ms")


Max Depth: 7,145
P1 Runtime: 26.17ms


In [153]:
# Part 2: Identify how many tiles are fully encircled by the main loop.


"""
Utility Functions
"""

# Take a list, and intersperse a value between each value in the list.
# I. E., intersperse([1, 2, 3], 0) -> [0, 1, 0, 2, 0, 3, 0]
def intersperse(l, filler):
    return [filler] if len(l) == 0 else [filler, l[0]] + intersperse(l[1:], filler)

# Convenient visibility tool.
def map_to_text(m):
    return "\n".join(map(lambda x: "".join(x), m))


"""
Execution
"""

# Since recursion makes this Jupyter Notebook hit the stack limit, use a depth-first-search back to the start by using the depth matrix to backtrack to build a boolean mask for which tiles are in the main loop.
rows, cols = len(search_matrix), len(search_matrix[0])
main_loop_matrix = [[False for _ in row] for row in maze_map]
mark_positions = loop_end
while len(mark_positions) > 0:
    (x, y, d), mark_positions = mark_positions[0], mark_positions[1:]
    main_loop_matrix[y][x] = True
    for nx, ny in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
        if in_bounds(nx, ny) and search_matrix[ny][nx] == d - 1:
            mark_positions.append((nx, ny, d - 1))

# Apply derived mask to the maze map to turn all non-main-loop characters into empty space.
main_loop_map = [[maze_map[y][x] if main_loop_matrix[y][x] else "." for x in range(cols)] for y in range(rows)]

# Expand the map with periods, to effectively make visible the spaces between pipes.
expanded_map = intersperse(list(map(lambda x: intersperse(x, "."), main_loop_map)), ["."] * (cols * 2 + 1))
rows, cols = len(expanded_map), len(expanded_map[0])

# Now that we've interspersed extra rows and columns of periods, use the connections of all the pipe characters to fill their gaps with an X, blocking our flood fill later.
filled_map = [[c for c in row] for row in expanded_map]
for y, row in enumerate(expanded_map):
    for x, char in enumerate(row):
        if char in ".XS":
            continue
        for cx, cy in connections_map[char]:
                filled_map[y + cy][x + cx] = "X"

# Once again resorting to a queue to get around Jupyter's low stack limit, flood fill all outside periods with spaces.
fill_queue, filled_positions = [(0, 0)], [[False for _ in row] for row in filled_map]
while len(fill_queue) > 0:
    (x, y), fill_queue = fill_queue[0], fill_queue[1:]
    filled_map[y][x] = " "
    for nx, ny in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
        if in_bounds(nx, ny) and filled_map[ny][nx] == "." and not filled_positions[ny][nx]:
            fill_queue.append((nx, ny))
            filled_positions[ny][nx] = True

# Return the map to its original size, now that we've filled all exterior space with spaces.
reshrunk_map = [[char for i, char in enumerate(row) if i % 2 != 0] for j, row in enumerate(filled_map) if j % 2 != 0]

# Count the number of periods remaining.
print(f"Enclosed Space: {sum([sum([1 if c == '.' else 0 for c in row]) for row in reshrunk_map])}")
print(f"Total Runtime: {1000*(time() - start_time):.2f}ms")

Enclosed Space: 445
Total Runtime: 126.98ms
